# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     | Adrian Marcelo Ballesteros Herrera  |
| **Fecha**      | 10/09/2025  |
| **Expediente** | 750743  |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

**Este modelo es el que no tiene penalizacion**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm


In [2]:
df=pd.read_csv(r"C:\Users\marce\OneDrive - ITESO\Quinto semestre\Lab estadistico\Advertising (1).csv")

In [3]:
df.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [4]:
df.describe()

,Unnamed: 0,TV,radio,newspaper,sales
count,200.000000,200.000000,200.000000,200.000000,200.000000
mean,100.500000,147.042500,23.264000,30.554000,14.022500
std,57.879185,85.854236,14.846809,21.778621,5.217457
min,1.000000,0.700000,0.000000,0.300000,1.600000
25%,50.750000,74.375000,9.975000,12.750000,10.375000
50%,100.500000,149.750000,22.900000,25.750000,12.900000
75%,150.250000,218.825000,36.525000,45.100000,17.400000
max,200.000000,296.400000,49.600000,114.000000,27.000000


In [5]:
df=df.drop(columns='Unnamed: 0')
print(df.columns)

Index(['TV', 'radio', 'newspaper', 'sales'], dtype='object')


In [6]:
df.describe()

,TV,radio,newspaper,sales
count,200.000000,200.000000,200.000000,200.000000
mean,147.042500,23.264000,30.554000,14.022500
std,85.854236,14.846809,21.778621,5.217457
min,0.700000,0.000000,0.300000,1.600000
25%,74.375000,9.975000,12.750000,10.375000
50%,149.750000,22.900000,25.750000,12.900000
75%,218.825000,36.525000,45.100000,17.400000
max,296.400000,49.600000,114.000000,27.000000


In [7]:
cols=df.columns
cols=cols.drop('sales')
x_list = []

for col in cols:
    x_list.append(df[col].values.reshape(-1,1))

n=len(df)                                      
x=np.hstack(x_list)
y=df.sales
lr=LinearRegression()
lr.fit(x,y)
print(lr.score(x,y),
lr.coef_,lr.intercept_)

0.8972106381789521 [ 0.04576465  0.18853002 -0.00103749] 2.938889369459403


In [8]:
yM=np.mean(y)
TSS=np.sum((y-yM)**2) 
RSS=(lr.score(x,y)-1)*TSS
print(abs(RSS))
ols= sm.OLS(y,x)
results=ols.fit()
results.summary()

556.8252629021875


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  sales   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.982
Method:                 Least Squares   F-statistic:                              3566.
Date:                Thu, 11 Sep 2025   Prob (F-statistic):                   2.43e-171
Time:                        11:04:19   Log-Likelihood:                         -423.54
No. Observations:                 200   AIC:                                      853.1
Df Residuals:                     197   BIC:                                      863.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0538      0.001     40.507      0.000       0.051       0.056
x2             0.2222      0.009     23.595      0.000       0.204       0.241
x3             0.0168      0.007      2.517      0.013       0.004       0.030
==============================================================================
Omnibus:                        5.982   Durbin-Watson:                   2.038
Prob(Omnibus):                  0.050   Jarque-Bera (JB):                7.039
Skew:                          -0.232   Prob(JB):                       0.0296
Kurtosis:                       3.794   Cond. No.                         12.6
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

**Primero el modelo de ridge o el L2 con sus coeficientes, interceptos y $R^2$**

In [10]:
ridge=Ridge(alpha=1)
ridge.fit(x,y)

print(f"Coeficientes:{ ridge.coef_}Intercepto:{ridge.intercept_} R^2:{ridge.score(x,y)}")
xM=np.mean(x)
yM=np.mean(y)
n=len(y)
TSS=np.sum((y-yM)**2) 
RSS=abs((ridge.score(x,y)-1)*TSS)
yV=abs(RSS)/(n-2)
SE02=yV*(1/n + (xM**2)/np.sum((x-xM)**2))
SE12=yV/np.sum((x-xM)**2)
SE0= np.sqrt(SE02)
SE1=np.sqrt(SE12)

Coeficientes:[ 0.04576464  0.1885251  -0.00103629]Intercepto:2.938967458330149 R^2:0.8972106380074802


In [11]:
Boin_upper=ridge.intercept_+2*SE0
Boin_lower=ridge.intercept_-2*SE0
B1in_upper=ridge.coef_+(2*SE1)
B1in_lower=ridge.coef_-(2*SE1)
t=ridge.coef_/SE1
from scipy import stats
p_bj=2*(1-stats.t.cdf(np.abs(t),n-1))
p_bj

array([0.        , 0.        , 0.24670982])

esto nos da que no podemos rechazar nuestra Ho del ultimo coeficiente que es newspapaer, por lo que B3 no es distinto de 0, esta propiedad la hace ridge ya que vuelve mas pequeños los coeficientes

**Primero el modelo de Lasso o el L1 con sus coeficientes, interceptos y $R^2$**

In [12]:
lasso=Lasso(alpha=0.4)
lasso.fit(x,y)
print(f"Coeficientes:{ lasso.coef_}Intercepto:{lasso.intercept_} R^2:{lasso.score(x,y)}")
xM=np.mean(x)
yM=np.mean(y)
n=len(y)
TSS=np.sum((y-yM)**2) 
RSS=abs((lasso.score(x,y)-1)*TSS)
yV=abs(RSS)/(n-2)
SE02=yV*(1/n + (xM**2)/np.sum((x-xM)**2))
SE12=yV/np.sum((x-xM)**2)
SE0= np.sqrt(SE02)
SE1=np.sqrt(SE12)

Coeficientes:[ 0.04571746  0.18618229 -0.        ]Intercepto:2.9687448333814217 R^2:0.8971669511584446


In [13]:
Boin_upper=lasso.intercept_+2*SE0
Boin_lower=lasso.intercept_-2*SE0
B1in_upper=lasso.coef_+(2*SE1)
B1in_lower=lasso.coef_-(2*SE1)
t=lasso.coef_/SE1
from scipy import stats
p_bj=2*(1-stats.t.cdf(np.abs(t),n-1))
p_bj

array([0., 0., 1.])

y bueno aca sale 1 por que el ultimo coeficiente ya nos lo da como 0 por lo que ya quita el coeficiente $B_3$

comparando los 3 resultados, me convence mas el sistema OLS sin penalizaciones pero los otros metodos tambien son buenos para algunos casos, no estoy muy seguro si para este es el mas indicado